# Load Data

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
DATA_DIR = '/content/drive/My Drive/FraudTransformer'

Mounted at /content/drive


In [ ]:
data_path = f'{DATA_DIR}/fe_banking_login_20201201_20210531.csv'

In [ ]:
feature_path = f'{DATA_DIR}/modified_coef_final4th.csv'

In [ ]:
raw_data = pd.read_csv(data_path)

In [ ]:
feature_list = pd.read_csv(feature_path)

In [ ]:
features = feature_list["Unnamed: 0"].astype("str").tolist()

In [ ]:
selected_data = raw_data[features+['frd', 'eval']]

In [ ]:
selected_data.rename({'tmxrc_Possible VPN or Tunnel': 'tmxrc_Possible_VPN_or_Tunnel'}, axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
features[29] = 'tmxrc_Possible_VPN_or_Tunnel'

In [ ]:
selected_data.apply(pd.Series.value_counts)

,tmxrc_Aggregator,tmxrc_ACC_EMAIL_LCL_AGE_GT_3MTHS,tmxrc_ACC_LOGIN_SMART_ID_LCL_PAIR_AGE_GT_2MTHS,tmxrc_ACC_LOGIN_SMART_ID_LCL_PAIR_AGE_GT_MTH,tmxrc_SMART_ID_LCL_AGE_GT_3MTHS,tmxrc_ACC_EMAIL_SMART_ID_LCL_PAIR_AGE_GT_3MTHS,tmxrc_ACC_EMAIL_LCL_AGE_GT_MTH,tmxrc_AGENT_PUBLICKEY_HASH_LCL_AGE_GT_3MTHS,tmxrc_ACC_LOGIN_SMART_ID_LCL_PAIR_AGE_GT_2WKS,tmxrc_AGENT_PUBLICKEY_HASH_LCL_AGE_GT_MTH,...,tmxrc_ACC_EMAIL_GBL_AGE_LT_DY,tmxrc_ACC_NUM_SMART_ID_LCL_PAIR_AGE_LT_DY,tmxrc_Jailbreak_or_Root,tmxrc_Proxy_Anonymous,sumrc_Level_1_Link_Reject,tmxrc_Rejected_ExactID_Month,tmxrc_PotentialVM_UA_Android_OS_Win,tmxrc_Phones_per_ExactID_Week_High,frd,eval
0,915124,738032,858212,828697,409576,852116,925615,665783,872812,852589,...,931852,929281,929089,913852,906871,906677,931257,930973,801345,749142
1,19651,196743,76563,106078,525199,82659,9160,268992,61963,82186,...,2923,5494,5686,20923,27904,28098,3518,3802,133430,185633


In [ ]:
selected_data.iloc[0]

tmxrc_Aggregator                                      0
tmxrc_ACC_EMAIL_LCL_AGE_GT_3MTHS                      0
tmxrc_ACC_LOGIN_SMART_ID_LCL_PAIR_AGE_GT_2MTHS        0
tmxrc_ACC_LOGIN_SMART_ID_LCL_PAIR_AGE_GT_MTH          0
tmxrc_SMART_ID_LCL_AGE_GT_3MTHS                       0
tmxrc_ACC_EMAIL_SMART_ID_LCL_PAIR_AGE_GT_3MTHS        0
tmxrc_ACC_EMAIL_LCL_AGE_GT_MTH                        0
tmxrc_AGENT_PUBLICKEY_HASH_LCL_AGE_GT_3MTHS           0
tmxrc_ACC_LOGIN_SMART_ID_LCL_PAIR_AGE_GT_2WKS         0
tmxrc_AGENT_PUBLICKEY_HASH_LCL_AGE_GT_MTH             0
tmxrc_ACC_EMAIL_SMART_ID_LCL_PAIR_AGE_GT_2MTHS        0
tmxrc_ACC_NAME_LCL_AGE_GT_3MTHS                       0
tmxrc_ACC_EMAIL_SMART_ID_GBL_PAIR_AGE_GT_2MTHS        0
tmxrc_SMART_ID_LCL_AGE_GT_2WKS                        0
tmxrc_DIG_ID_GBL_AGE_GT_3MTHS                         0
tmxrc_DIG_ID_LCL_AGE_GT_3MTHS                         0
tmxrc_DIG_ID_LCL_AGE_LT_DY                            0
tmxrc_AGENT_PUBLICKEY_HASH_LCL_AGE_LT_DY        

In [ ]:
test_data = selected_data[selected_data['eval'] == True].iloc[:,0:46]

# Set Vocabulary and Tokenizer

In [ ]:
vocab_file = "vocab.txt"
## do not rerun this code chunk

In [ ]:
vocab =['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]','[BOS]','[EOS]']
for feature in features:
  vocab.append(feature+"_0")
  vocab.append(feature+"_1")
## do not rerun this code chunk

In [ ]:
with open(vocab_file, 'w') as vocab_fp:
  vocab_fp.write("\n".join(vocab))
## now the vocabulary is saved as vocab.txt

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.9 MB/s 
     |████████████████████████████████| 6.6 MB 70.0 MB/s 
     |████████████████████████████████| 120 kB 76.2 MB/s 


In [ ]:
from transformers import BertTokenizer

In [ ]:
special_tokens_dict = {"unk_token": "[UNK]", 
                       "sep_token": "[SEP]", 
                       "pad_token": "[PAD]", 
                       "cls_token": "[CLS]", 
                       "mask_token": "[MASK]",
                       "eos_token": "[EOS]",
                       "bos_token": "[BOS]"}

In [ ]:
tokenizer = BertTokenizer(vocab_file,do_lower_case=False, do_basic_tokenize = False)

In [ ]:
tokenizer.add_special_tokens(special_tokens_dict)

0

# Load Model

In [ ]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 5.2 MB/s 


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.1 MB/s 
     |████████████████████████████████| 115 kB 59.3 MB/s 
     |████████████████████████████████| 212 kB 59.9 MB/s 
     |████████████████████████████████| 127 kB 48.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from transformers import BertForSequenceClassification

In [ ]:
finetunemodel = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/checkpoint-98325-finetuned', num_labels=2)

In [ ]:
import transformers
import datasets
import shap
import numpy as np

In [ ]:
classifier = transformers.pipeline("sentiment-analysis", model=finetunemodel, tokenizer=tokenizer, return_all_scores=True)

`return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality


In [ ]:
# define the explainer
explainer = shap.Explainer(classifier)

# Streamlit

In [ ]:
!pip -q install streamlit
!pip install pyngrok==4.1.1

     |████████████████████████████████| 9.1 MB 7.0 MB/s 
     |████████████████████████████████| 235 kB 78.5 MB/s 
     |████████████████████████████████| 164 kB 90.0 MB/s 
     |████████████████████████████████| 4.7 MB 48.1 MB/s 
     |████████████████████████████████| 78 kB 8.3 MB/s 
     |████████████████████████████████| 181 kB 89.6 MB/s 
     |████████████████████████████████| 63 kB 734 kB/s 
     |████████████████████████████████| 51 kB 7.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=2e7b89c082b363b514ed1f3b4b9273944b7e2b4840401aed31a9a6583bf3aa9d
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok


In [ ]:
!ngrok authtoken 2DDjAAwoRbDxWnRkNabUZGjYWmy_4t1tcU9DFoqpyqYycXfgf

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 7.6 MB/s 
     |████████████████████████████████| 212 kB 73.6 MB/s 
     |████████████████████████████████| 115 kB 70.1 MB/s 
     |████████████████████████████████| 127 kB 88.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 5.3 MB/s 


In [ ]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 7.4 MB/s 


In [ ]:
%%writefile app.py
import streamlit as st

def intro():
    import streamlit as st

    st.write("# Fraud Detection with Deep Learning")
    st.sidebar.success("Select a demo above.")

    st.markdown(
        """This app uses BERT from the Hugging Face library library to detect frauds.
    """
    )

def single_demo():
  import numpy as np
  import pandas as pd
  import transformers
  from matplotlib import pyplot as plt
  import shap
  from transformers import BertForSequenceClassification
  from transformers import BertTokenizer
  import streamlit.components.v1 as components
  from IPython.core.interactiveshell import InteractiveShell
  from IPython.utils import capture

  # Load Data
  DATA_DIR = '/content/drive/My Drive/FraudTransformer'
  data_path = f'{DATA_DIR}/fe_banking_login_20201201_20210531.csv'
  feature_path = f'{DATA_DIR}/modified_coef_final4th.csv'
  raw_data = pd.read_csv(data_path)
  feature_list = pd.read_csv(feature_path)
  features = feature_list["Unnamed: 0"].astype("str").tolist()
  selected_data = raw_data[features+['frd', 'eval']]
  selected_data.rename({'tmxrc_Possible VPN or Tunnel': 'tmxrc_Possible_VPN_or_Tunnel'}, axis=1, inplace=True)
  features[29] = 'tmxrc_Possible_VPN_or_Tunnel'
  test_data = selected_data[selected_data['eval'] == True].iloc[:,0:46]

  # Set Tokenizer
  vocab_file = "vocab.txt"
  special_tokens_dict = {"unk_token": "[UNK]", 
                        "sep_token": "[SEP]", 
                        "pad_token": "[PAD]", 
                        "cls_token": "[CLS]", 
                        "mask_token": "[MASK]",
                        "eos_token": "[EOS]",
                        "bos_token": "[BOS]"}
  tokenizer = BertTokenizer(vocab_file,do_lower_case=False, do_basic_tokenize = False)
  tokenizer.add_special_tokens(special_tokens_dict)

  # Load Model
  finetunemodel = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/checkpoint-98325-finetuned', num_labels=2)
  classifier = transformers.pipeline("sentiment-analysis", model=finetunemodel, tokenizer=tokenizer, return_all_scores=True)
  explainer = shap.Explainer(classifier)

  def st_plot_text_shap(shap_val, height=None):
      InteractiveShell().instance()
      with capture.capture_output() as cap: 
          shap.plots.text(shap_val[:,:,"LABEL_1"])
      components.html(cap.outputs[0].data['text/html'], height=height, scrolling=True)

  st.markdown(f'# {list(page_names_to_funcs.keys())[1]}')

  # Promt Input
  st.markdown("Now select a row from the test set")
  max_row = len(test_data)-1
  index= st.number_input(
              "Pick up a row", min_value=0, max_value=max_row, step=1, value=0, key='singleinput')
  input_list = test_data.iloc[index]
  input_df = test_data.iloc[[index]]
  show_row = st.button("Show selected row")
  if show_row:
    st.text("Your input is:")
    st.dataframe(input_df)

  st.subheader("Fraud Classification")
  run_model = st.button("Run BERT model")
  if run_model:
    input_list = test_data.iloc[index]
    test_data_features = input_list[0:45]
    test_data_labels = input_list[45]
    x = test_data_features.to_string(header=False).split('\n')
    vals = ['_'.join(ele.split()) for ele in x]
    val = " ".join(vals)
    st.text('{}'.format(classifier(val)))
    st.text('{}'.format(input_list[45]))

    st.subheader("SHAP values plot")
    shap_values = explainer([val])
    st_plot_text_shap(shap_values)

def dataset_demo():
  import numpy as np
  import pandas as pd
  import transformers
  from transformers import BertForSequenceClassification
  from transformers import BertTokenizer
  from transformers import pipeline
  from datasets import Dataset
  import torch
  
  st.write('Now upload your own fraud detection dataset with, of course, the same format')
  uploaded_file = st.file_uploader(
        "Choose a CSV file", type="csv")
  if uploaded_file is not None:
    run_model = st.button("Run ML model")
    if run_model:
      test_data = pd.read_csv(uploaded_file)
      DATA_DIR = '/content/drive/My Drive/FraudTransformer'
      feature_path = f'{DATA_DIR}/modified_coef_final4th.csv'
      feature_list = pd.read_csv(feature_path)
      features = feature_list["Unnamed: 0"].astype("str").tolist()
      features[29] = 'tmxrc_Possible_VPN_or_Tunnel'

      vocab_file = "vocab.txt"
      special_tokens_dict = {"unk_token": "[UNK]", 
                            "sep_token": "[SEP]", 
                            "pad_token": "[PAD]", 
                            "cls_token": "[CLS]", 
                            "mask_token": "[MASK]",
                            "eos_token": "[EOS]",
                            "bos_token": "[BOS]"}
      tokenizer = BertTokenizer(vocab_file,do_lower_case=False, do_basic_tokenize = False)
      tokenizer.add_special_tokens(special_tokens_dict)

      test_data_features = test_data.iloc[:,0:45]
      test_data_labels = test_data.iloc[:,45]
      vocab_dict = tokenizer.get_vocab()

      for column in test_data_features:
        test_data_features[column] = test_data_features[column].map(lambda x:column + "_" + str(x))
        test_data_features[column] = test_data_features[column].map(lambda x: vocab_dict.get(x, vocab_dict['[UNK]']))
      input_initial = test_data_features.to_numpy().tolist()
      input = [[3]+x+[1] for x in input_initial]
      my_dict = {'input_ids': input, 
           'labels': test_data_labels.to_list(), 
           'token_type_ids': [[0]*(len(test_data_features.columns)+2)]*len(test_data_features),
           'attention_mask': [[1]*(len(test_data_features.columns)+2)]*len(test_data_features)}
      test_df = Dataset.from_dict(my_dict)
      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
      test_df = test_df.with_format("torch", device=device)
      finetunemodel = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/checkpoint-98325-finetuned', num_labels=2).to("cuda")
      finetunemodel.eval()
      from torch.utils.data import TensorDataset, DataLoader, SequentialSampler
      val_data = TensorDataset(test_df['input_ids'], test_df['attention_mask'], test_df['labels'])
      val_sampler = SequentialSampler(val_data)
      val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=32)
      import torch.nn.functional as F
      def bert_predict(model, test_dataloader):
          """Perform a forward pass on the trained BERT model to predict probabilities
          on the test set.
          """
          # Put the model into the evaluation mode. The dropout layers are disabled during
          # the test time.
          model.eval()

          all_logits = []

          # For each batch in our test set...
          for batch in test_dataloader:
              # Load batch to GPU
              b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

              # Compute logits
              with torch.no_grad():
                  logits = model(b_input_ids, b_attn_mask)['logits']
              all_logits.append(logits)
          
          # Concatenate logits from each batch
          all_logits = torch.cat(all_logits, dim=0)

          # Apply softmax to calculate probabilities
          probs = F.softmax(all_logits, dim=1).cpu().numpy()

          return probs
      probs = bert_predict(finetunemodel, val_dataloader)
      from sklearn.metrics import roc_auc_score
      import numpy as np
      y_true = np.array(test_data.iloc[:,45])
      y_scores = probs[:,1]
      st.text('{}'.format(roc_auc_score(y_true, y_scores)))


page_names_to_funcs = {
    "—": intro,
    "Single Demo": single_demo,
    "Dataset Demo": dataset_demo
}

demo_name = st.sidebar.selectbox("Choose a demo", page_names_to_funcs.keys())
page_names_to_funcs[demo_name]()

Overwriting app.py


In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(port='80')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null

http://8847-34-83-10-82.ngrok.io
2022-09-08 17:02:02.125 INFO    numexpr.utils: NumExpr defaulting to 4 threads.
tcmalloc: large alloc 1346076672 bytes == 0xb1db4000 @  0x7fc2f19001e7 0x7fc2ee4f00ce 0x7fc2ee546cf5 0x7fc2ee546f4f 0x7fc2ee5e9673 0x5936cc 0x548c51 0x5127f1 0x593dd7 0x5118f8 0x549576 0x593fce 0x548ae9 0x5127f1 0x593dd7 0x5118f8 0x549576 0x593fce 0x548ae9 0x5127f1 0x549e0e 0x593fce 0x548ae9 0x5127f1 0x549e0e 0x593fce 0x548ae9 0x5127f1 0x549e0e 0x4bca8a 0x532b86

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
`return_all_scores` is now deprecated, use `top_k=1` if you want similar functionnality
tcmalloc: large alloc 1346076672 bytes == 0x122b18000 @  0x7fc2f19001e7 0x7fc2ee4f00ce 0x7fc2ee546cf5 0x7fc2ee546f4f 0x7fc2ee5e9673 0x5936cc 0x548c51 0x5127f1 0x593dd7 0x5118f8 0x549576 0x593fce 0x548ae9 0x5127f1 0x593dd7 0x511

# Testing

In [ ]:
import numpy as np
import pandas as pd
import streamlit as st
import transformers
import datasets
from matplotlib import pyplot as plt
import matplotlib
from datasets import Dataset
import shap
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import streamlit.components.v1 as components
from IPython.core.interactiveshell import InteractiveShell
from IPython.utils import capture

# Load Data
DATA_DIR = '/content/drive/My Drive/FraudTransformer'
data_path = f'{DATA_DIR}/fe_banking_login_20201201_20210531.csv'
feature_path = f'{DATA_DIR}/modified_coef_final4th.csv'
raw_data = pd.read_csv(data_path)
feature_list = pd.read_csv(feature_path)
features = feature_list["Unnamed: 0"].astype("str").tolist()
selected_data = raw_data[features+['frd', 'eval']]
selected_data.rename({'tmxrc_Possible VPN or Tunnel': 'tmxrc_Possible_VPN_or_Tunnel'}, axis=1, inplace=True)
features[29] = 'tmxrc_Possible_VPN_or_Tunnel'
test_data = selected_data[selected_data['eval'] == True].iloc[:,0:46]


# Set Tokenizer
vocab_file = "vocab.txt"
special_tokens_dict = {"unk_token": "[UNK]", 
                       "sep_token": "[SEP]", 
                       "pad_token": "[PAD]", 
                       "cls_token": "[CLS]", 
                       "mask_token": "[MASK]",
                       "eos_token": "[EOS]",
                       "bos_token": "[BOS]"}
tokenizer = BertTokenizer(vocab_file,do_lower_case=False, do_basic_tokenize = False)
tokenizer.add_special_tokens(special_tokens_dict)

# Load Model
finetunemodel = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/checkpoint-98325-finetuned', num_labels=2)
classifier = transformers.pipeline("sentiment-analysis", model=finetunemodel, tokenizer=tokenizer, return_all_scores=True)
explainer = shap.Explainer(classifier)


In [ ]:
index = 0
input_list = test_data.iloc[index]
input_df = test_data.iloc[[index]]
test_data_features = input_list[0:45]
test_data_labels = input_list[45]
x = test_data_features.to_string(header=False).split('\n')
vals = ['_'.join(ele.split()) for ele in x]
val = " ".join(vals)

shap_values = explainer([val])


  0%|          | 0/498 [00:00<?, ?it/s]

In [ ]:
shap.plots.text(shap_values[:,:,"LABEL_1"])

In [ ]:
def st_plot_text_shap(shap_val, height=None):
    InteractiveShell().instance()
    with capture.capture_output() as cap: 
        shap.plots.text(shap_val)
    components.html(cap.outputs[0].data['text/html'], height=height, scrolling=True)

In [ ]:
InteractiveShell().instance()
with capture.capture_output() as cap: 
  shap.plots.text(shap_values[:,:,"LABEL_1"])